In [128]:
import json
import geopandas as gpd
import pandas as pd
import ast

# Facility Data

In [124]:
def data_to_json(filepath:str)->dict:
    with open(filepath, "r") as f:
        t = f.read()
    js = json.loads(t)
    return js


In [79]:
#Important attributes within facility json data
item_list = ['uic', 'name','location','unit','roletype','capabilities', 'subjectiveStatusLevelName',
             'subjectiveStatusLevelColor','subjectiveStatusLevelListOrder',
             'component','countryCode','par','itemCategories']

In [80]:
facility_data = []
for item in js:
    attrs={}
    for key in item.keys():
        if key in item_list:
            attrs[key] = item[key]
    facility_data.append(attrs)

In [82]:
df = pd.DataFrame(facility_data)

# MEDCOP Unit Locations

In [127]:
medcop_json = data_to_json(filepath='./sample_JSON/medcop_unit_locations_1000.json.txt')

In [87]:
medcop_gdf = gpd.GeoDataFrame.from_features(medcop_json)

In [90]:
medcop_attrs_drop_list = ['abbrname', 'last_updated', 'address',
                          'last_modified_by', 'phone', 'unit_uic', 'icon_url',
                          'classification','facility_tags_agg',
                          'on_hand_item_type_names_agg','unit_name', 'component',
                           'parent_commands', 'parent_jtfs_flat', 'last_medsit', 'last_medsit_by',
                            'subjective_status_classification', 'subjective_status_updated_by',
                            'subjective_status_updated']

In [91]:
medcop_filt = medcop_gdf.drop(columns=medcop_attrs_drop_list)

In [97]:
def unpack_dict_fields(df: pd.DataFrame,column_name: str) -> pd.DataFrame:
    copy_df = df.copy()
    for idx, row in copy_df.iterrows():
        if row[column_name]:
            for item in ast.literal_eval(row[column_name]):
                for key in item:
                    copy_df.at[idx,key] = item[key]

    return copy_df

def unpack_list_fields(df: pd.DataFrame,column_name: str) -> pd.DataFrame:
    copy_df = df.copy()
    for idx, row in copy_df.iterrows():
        if row[column_name]:
            for item in ast.literal_eval(row[column_name]):
                copy_df.at[idx,item] = 'True'

    return copy_df

In [98]:
bed_df = unpack_dict_fields(all_df,column_name='beds_status_fields')
on_hand_df = unpack_list_fields(all_df,column_name='on_hand_item_type_names')
equip_status_df = unpack_dict_fields(all_df,column_name='equipment_status_fields')
supply_status_df = unpack_dict_fields(all_df,column_name='supplies_status_fields')
person_status_df = unpack_dict_fields(all_df,column_name='personnel_status_fields')
lab_status_df = unpack_dict_fields(all_df,column_name='labs_status_fields')
surveil_status_df = unpack_dict_fields(all_df,column_name='surveillance_status_fields')
vaccine_status_df = unpack_dict_fields(all_df,column_name='vaccines_status_fields')



/var/folders/j9/x5yr8_d15r75klcg1_xt7k900000gn/T/ipykernel_14279/2813300213.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  copy_df.at[idx,item] = 'True'
/var/folders/j9/x5yr8_d15r75klcg1_xt7k900000gn/T/ipykernel_14279/2813300213.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  copy_df.at[idx,item] = 'True'
/var/folders/j9/x5yr8_d15r75klcg1_xt7k900000gn/T/ipykernel_14279/2813300213.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  copy_df.at[id

# Medical Facs

In [129]:
med_facs = data_to_json('./sample_JSON/Medical_facilities.json.txt')

# Mission Data

In [133]:
mission_data = data_to_json('./sample_JSON/Mission_1016.json.txt')

In [134]:
mission_data

{'id': 'cfa31661-6523-4d06-bed5-d0b0777ad04b',
 'missionNumber': '1016',
 'missionStatusID': '3001da17-d010-4ff8-aa2e-106ddd7898e6',
 'missionRequestTypeID': '38388cba-e15b-47dc-b9c2-e4bdae62f3ba',
 'missionTypeID': '3a9f5554-cf99-45aa-a090-885a3e0d24af',
 'cocom': 'INDOPACOM',
 'receivingFacilityUIC': 'WBH3AA1',
 'created': '2024-03-15T16:11:06.5641Z',
 'createdBy': 'users.cce.af.mil\\CHARLES.L.COOK.1088526313',
 'updated': '2024-03-15T17:36:58.249799Z',
 'updatedBy': 'users.cce.af.mil\\JOSE.A.CARDENAS.1374425591',
 'classification': {'level': 'CUI',
  'aea': {'value': ''},
  'classificationString': 'CUI - (WARNING - Training System With Notional Data - DO NOT put Operational Data into this environment)',
  'portionString': '(CUI) - (WARNING - Training System With Notional Data - DO NOT put Operational Data into this environment)'},
 'groups': ['public'],
 'missionStatusName': 'Completed',
 'missionTypeCode': 'POI',
 'missionTypeName': 'Point of Injury',
 'missionRequestTypeName': 'US